In [152]:

#Making all necessary imports for the code
from datetime import date
from jugaad_data.nse import bhavcopy_save

#Saving the Bhavcopy file for 01-01-2021
bhavcopy_save(date(2023,6,1), ".")


'.\\cm01Jun2023bhav.csv'

In [153]:
import pandas as pd
import numpy as np

df=pd.read_csv("stockdata.csv")

In [154]:
df.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,Unnamed: 13
0,182D021123,TB,96.77,96.77,96.77,96.77,96.77,96.47,1000,96770.00,01-JUN-2023,1,IN002023Y052,NaN
1,182D220923,TB,97.60,97.60,97.60,97.60,97.60,97.60,1100,107360.00,01-JUN-2023,1,IN002022Y526,NaN
2,182D231123,TB,96.60,96.60,96.60,96.60,96.60,96.60,4800,463680.00,01-JUN-2023,1,IN002023Y086,NaN
3,20MICRONS,EQ,92.30,94.20,92.10,92.65,92.45,92.25,126687,11809517.10,01-JUN-2023,2672,INE144J01027,NaN
4,21STCENMGM,EQ,17.85,18.20,17.75,17.90,18.15,17.85,102,1829.55,01-JUN-2023,12,INE253B01015,NaN


In [155]:
df_fo=pd.read_csv("fo_mktlots.csv")

In [156]:
df_fo.head()

,UNDERLYING,SYMBOL,JUL-23,AUG-23,SEP-23,DEC-23,MAR-24,JUN-24,DEC-24,JUN-25,DEC-25,JUN-26,DEC-26,JUN-27,DEC-27,JUN-28
0,NIFTY FINANCIAL SERVICES,FINNIFTY,40,40,40,,,,,,,,,,,
1,NIFTY MID SELECT,MIDCPNIFTY,75,75,75,,,,,,,,,,,
2,NIFTY BANK,BANKNIFTY,25,15,15,15,15,15,,,,,,,,
3,NIFTY 50,NIFTY,50,50,50,50,50,50,50,50,50,50,50,50,50,50
4,Derivatives on Individual Securities,Symbol,JUL-23,AUG-23,SEP-23,,,,,,,,,,,


In [157]:
df['SYMBOL'] = df['SYMBOL'].str.strip()
df_fo['SYMBOL    '] = df_fo['SYMBOL    '].str.strip()
common_words = set(df['SYMBOL']).intersection(df_fo['SYMBOL    '])

In [158]:
num_common_words=len(common_words)
print(num_common_words)

186


In [159]:
data=df[df['SYMBOL'].isin(common_words)]

In [160]:
data.to_csv('data.csv',index=False)

In [161]:
def get_top5performers(data):
    rolling_returns = data["CLOSE"].pct_change(52).rolling(52).mean()
    top5_performers = rolling_returns.sort_values(ascending=False).head(5)
    top5_df = data.loc[top5_performers.index]
    top5_df = top5_df.sort_values(by="CLOSE", ascending=False)
    return top5_df

In [162]:
top5_performers= get_top5performers(data)

top5_performers

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,Unnamed: 13
1580,OFSS,EQ,3650.00,3656.00,3610.00,3616.55,3621.00,3633.60,28380,1.032895e+08,01-JUN-2023,5138,INE881D01027,NaN
1569,NTPC,ND,1226.00,1226.00,1211.66,1219.65,1211.66,1275.00,157,1.914852e+05,01-JUN-2023,5,INE733E07JW2,NaN
1578,OBEROIRLTY,EQ,930.75,952.90,928.00,940.55,941.60,930.75,674688,6.362893e+08,01-JUN-2023,28325,INE093I01010,NaN
1591,ONGC,EQ,154.70,156.00,153.20,153.65,154.00,154.90,13982606,2.156090e+09,01-JUN-2023,78938,INE213A01029,NaN
1568,NTPC,N7,10.30,10.34,10.30,10.32,10.33,10.30,11634,1.200052e+05,01-JUN-2023,72,INE733E07JP6,NaN


In [163]:
top5_performers.index

Int64Index([1580, 1569, 1578, 1591, 1568], dtype='int64')

In [164]:
import talib

def calculate_atr(data):
    atr = talib.ATR(
        data["HIGH"], data["LOW"], data["CLOSE"], timeperiod=14,
    )
    return atr

def go_long(data, top5_performers):
    atr = calculate_atr(data)
    symbols = top5_performers.index.to_list()
    positions = pd.DataFrame(
        columns=["Symbol", "Price", "Stop Loss"], index=symbols
    )
    for stock in symbols:
        price = data.loc[stock, "CLOSE"]
        atr_for_stock = atr[stock]
        stop_loss = price - atr_for_stock * 2
        positions.loc[stock, :] = [stock, price, stop_loss]
    return positions

In [165]:
positions=go_long(data,top5_performers)
positions

,Symbol,Price,Stop Loss
1580,1580,3616.55,-11839.585145
1569,1569,1219.65,-16211.04224
1578,1578,940.55,-15286.75708
1591,1591,153.65,-14693.239778
1568,1568,10.32,-18574.167027


In [166]:
# Rebalance the stocks on weekly basis
def rebalance(positions):
    new_positions = []
    for stock, price, stop_loss in positions:
        if data.loc[stock, "CLOSE"] > stop_loss:
            new_positions.append((stock, price, stop_loss))
    return new_positions

In [167]:
import pyfolio as pf
pf.create_returns_tear_sheet(top5_performers)

C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\empyrical\utils.py:440: UserWarning: Yahoo Finance read failed: mktime argument out of range, falling back to Google
  warnings.warn(


AttributeError: module 'pandas_datareader.data' has no attribute 'get_data_google'